In [76]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import os
import re
import time
import numpy as np

In [118]:
def scrape_articles(journal, start, end, times):
    
    email = 'xianlin.ding@sciencespo.fr'
    password = '099115Keep12?'
    path = os.getcwd()
    path = re.sub('Code','Result',path)
    
    # go into the Eruopresse website
    url = "https://acces-distant.sciencespo.fr/fork?https://nouveau.europresse.com/access/ip/default.aspx?un=politique2T_1"
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.find_element(By.ID, 'username').send_keys(email)
    driver.find_element(By.ID, 'password').send_keys(password)
    time.sleep(3)
    driver.find_element(By.CLASS_NAME, 'btn-submit').click()
    
    # choose the criteria of searching
    driver.find_element(By.CLASS_NAME, 'lnk-text').click()
    time.sleep(3)
    driver.find_element(By.ID, 'specific-sources-rd').click()
    time.sleep(5)
    
    # choose the journal
    try:
        driver.find_element(By.ID, journal).click()
    except:
        time.sleep(20)
        driver.find_element(By.ID, journal).click()
    
    # key words : climat and climatique
    driver.find_element(By.XPATH, '//textarea').send_keys('climat') 
    driver.find_elements(By.CLASS_NAME, 'criteria-oper-lbl')[1].click()
    Select(driver.find_element(By.NAME, 'CriteriaKeys[0].Key')).select_by_value('TEXT')
    driver.find_element(By.NAME, 'CriteriaKeys[0].Text').send_keys('climatique')
    
    # choose the searching period
    Select(driver.find_element(By.ID, 'DateFilter_DateRange')).select_by_value('10')

    Select(driver.find_elements(By.CLASS_NAME, 'year')[1]).select_by_value(start[0])
    Select(driver.find_elements(By.CLASS_NAME, 'year')[2]).select_by_value(end[0])

    Select(driver.find_elements(By.CLASS_NAME, 'month')[1]).select_by_value(start[1])
    Select(driver.find_elements(By.CLASS_NAME, 'month')[2]).select_by_value(end[1])

    Select(driver.find_elements(By.CLASS_NAME, 'day')[1]).select_by_value(start[2])
    Select(driver.find_elements(By.CLASS_NAME, 'day')[2]).select_by_value(end[2])
    
    driver.find_element(By.ID, 'btnSearch').click()
    
    # scroll page
    for i in range(times):
        temp_height=0
        while True:
            driver.execute_script("window.scrollBy(0,1000)")
            check_height = driver.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
            if check_height==temp_height:
                break
            temp_height=check_height
        
        time.sleep(5)
        if (i+1) % 5 == 0:
            print('scrolling ', i+1, ' times' )
        
    # get web contents
    web_content = driver.page_source
    obj = BeautifulSoup(web_content,'lxml').body
    obj = obj.aside.ul
    obj = obj.find_all(name='div', attrs={"class":"docListItem msDocItem"})
    
    # start to build data
    info = pd.DataFrame(columns = ['date', 'publication', 
                                   'words', 'author',
                                   'title', 'intro'])
    
    i=0
    for obj_i in obj:
        
        obj_i = obj_i.div.next_sibling.next_sibling
        
        # publication
        try:
            obj_i_1 = obj_i.div
        except:
            print('Article ' + i + ' goes wrong')
            continue
        
        obj_i_1 = obj_i
        
        obj_i_1 = obj_i_1.find_all(name='span', attrs={"class":"source-name"})[0]
        obj_i_1 = str(obj_i_1)
        publication_i = re.findall(r"<span class=\"source-name\">(.+?)</span>", obj_i_1)[0]
        
        # title
        obj_i_2 = obj_i.div.next_sibling.next_sibling
        title_i = obj_i_2.div.a
        title_i = str(title_i)
        title_i = re.findall(r">(.+?)</a>", title_i)[0]
        title_i = re.sub('\u200a','',title_i)
        
        # date and words
        obj_i_3 = obj_i_2.div.next_sibling.next_sibling 
        
        date_words_i = obj_i_3.div
        date_words_i = str(date_words_i)
        date_i = re.findall(r"<span class=\"details\">(.+?)<span class=", date_words_i)[0]
        date_i = date_i.replace(' ', '')
        
        words_i = re.findall(r"</span>(.+?)</span>", date_words_i)[0]
        words_i = words_i.replace(' ', '')
        
        # authors and introduction
        obj_i_4 = obj_i_3.div.next_sibling.next_sibling
        obj_i_4 = str(obj_i_4)
        
        try:
            obj_i_4 = obj_i_4.split('<span class="doclist-author">')[1]
            author_i = obj_i_4.split('</span>')[0]
            intro_i = obj_i_4.split('</span>')[1]
        
        except:
            author_i = None
            intro_i = obj_i_4.split('</span> </span>')[1]
    
        intro_i = intro_i.split('</div>')[0]
        intro_i = re.sub(r"<([a-z]+)>",'',intro_i)
        intro_i = re.sub(r"</([a-z]+)>",'',intro_i)
        intro_i = re.sub('\u200a','',intro_i)
        intro_i = re.sub('-','',intro_i)
        
        info.loc[i] = [date_i, publication_i, 
                       words_i, author_i,
                       title_i, intro_i]
        i = i+1
    
    info_name = str(publication_i) + '_' + \
              str(start[0]) + '_' + \
              str(start[1]) + '_' + str(end[1])
    
    info.to_csv(path + '\\data_raw\\'+ info_name + '.csv')
    
    print('Import Data ' + info_name + 'finished.')
    
    return None

In [78]:
def generate_period(year):
    
    start_md = [['1','1'],['5','1'],['9','1']]
    end_md = [['4','30'], ['8','31'], ['12','31']]
    
    start = []
    for i in year:    
        for j in start_md:
            start_ij = []
            start_ij.append(i)
            start_ij = start_ij + j
            start.append(start_ij)
            
    end = []
    for i in year:    
        for j in end_md:
            end_ij = []
            end_ij.append(i)
            end_ij = end_ij + j
            end.append(end_ij)
            
    return start,end

In [79]:
# To avoid missing data during collection, 
# We split it into several parts 
# by journals 

In [80]:
## Le monde : journal_list = 'sf_247'

## last five years 
## the first one is an example
year_list = list(map(lambda x:str(x),list(range(2018,2023))))
start_list, end_list = generate_period(year_list)

for i in range(5):
    
    for j in range(3):
        
        journal = 'sf_247'
        start = start_list[i*3 + j]
        end = end_list[i*3 + j]
        times = 35
        
        scrape_articles(journal, start, end, times)


<ipython-input-77-3b72dbe67953>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
scrolling  30  times
scrolling  35  times
Import Data Le Monde_2018_1_4finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
scrolling  30  times
scrolling  35  times
Import Data Le Monde_2018_5_8finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
scrolling  30  times
scrolling  35  times
Import Data Le Monde_2018_9_12finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
scrolling  30  times
scrolling  35  times
Import Data Le Monde_2019_1_4finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
scrolling  30  times
scrolling  35  times
Import Data Le Monde_2019_5_8finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  2

In [82]:
# the data for Le Monde_2022_9_12  is not fully scraped (more than 1000 articles)
# re-do it
scrape_articles(journal, ['2022', '9', '1'], ['2022', '10', '31'], times)
scrape_articles(journal, ['2022', '11', '1'], ['2022', '12', '31'], times)

<ipython-input-77-3b72dbe67953>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
scrolling  30  times
scrolling  35  times
Import Data Le Monde_2022_9_10finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
scrolling  30  times
scrolling  35  times
Import Data Le Monde_2022_11_12finished.


In [83]:
# now change it into functions
def import_data(journal_code, year_list):
    
    start_list, end_list = generate_period(year_list)
    
    for i in range(len(year_list)):
        for j in range(3):
            
            journal = journal_code
            start = start_list[i*3 + j]
            end = end_list[i*3 + j]
            times = 25
            scrape_articles(journal, start, end, times)

In [84]:
# continue import data
## Le monde, 2013-2017
## condition of networks went wrong when craping data of 2017

journal_code = 'sf_247'
year_list = list(map(lambda x:str(x),list(range(2013,2017))))

import_data(journal_code, year_list)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.78M/6.78M [00:00<00:00, 11.1MB/s]
<ipython-input-77-3b72dbe67953>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
Import Data Le Monde_2013_1_4finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
Import Data Le Monde_2013_5_8finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
Import Data Le Monde_2013_9_12finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
Import Data Le Monde_2014_1_4finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
Import Data Le Monde_2014_5_8finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
Import Data Le Monde_2014_9_12finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
Import Data Le Monde_2015_1_4finishe

UnexpectedAlertPresentException: Alert Text: Le nombre maximum de connexions simultanées a été atteint. Veuillez recommencer plus tard ou contacter votre administrateur.
Message: unexpected alert open: {Alert text : Le nombre maximum de connexions simultanées a été atteint. Veuillez recommencer plus tard ou contacter votre administrateur.}
  (Session info: chrome=110.0.5481.96)
Stacktrace:
Backtrace:
	(No symbol) [0x009537D3]
	(No symbol) [0x008E8B81]
	(No symbol) [0x007EB36D]
	(No symbol) [0x008518E5]
	(No symbol) [0x0083B216]
	(No symbol) [0x00810D97]
	(No symbol) [0x0081253D]
	GetHandleVerifier [0x00BCABF2+2510930]
	GetHandleVerifier [0x00BF8EC1+2700065]
	GetHandleVerifier [0x00BFC86C+2714828]
	GetHandleVerifier [0x00A03480+645344]
	(No symbol) [0x008F0FD2]
	(No symbol) [0x008F6C68]
	(No symbol) [0x008F6D4B]
	(No symbol) [0x00900D6B]
	BaseThreadInitThunk [0x75B900F9+25]
	RtlGetAppContainerNamedObjectPath [0x77457BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77457B8E+238]


In [85]:
## Le monde, 2017
## redo it again due to network condition

journal_code = 'sf_247'
year_list = list(map(lambda x:str(x),list(range(2017,2018))))

import_data(journal_code, year_list)

<ipython-input-77-3b72dbe67953>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
Import Data Le Monde_2017_1_4finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
Import Data Le Monde_2017_5_8finished.
scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times
Import Data Le Monde_2017_9_12finished.


In [119]:
## Le Figaro
journal_code = 'sf_246'
year_list = list(map(lambda x:str(x),list(range(2013,2023))))

import_data(journal_code, year_list)

<ipython-input-118-70e3e1a016f1>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


scrolling  5  times
scrolling  10  times
scrolling  15  times
scrolling  20  times
scrolling  25  times


TypeError: can only concatenate str (not "int") to str

In [96]:
email = 'xianlin.ding@sciencespo.fr'
password = '099115Keep12?'
path = os.getcwd()
path = re.sub('Code','Result',path)
    
# go into the Eruopresse website
url = "https://acces-distant.sciencespo.fr/fork?https://nouveau.europresse.com/access/ip/default.aspx?un=politique2T_1"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.find_element(By.ID, 'username').send_keys(email)
driver.find_element(By.ID, 'password').send_keys(password)
time.sleep(3)
driver.find_element(By.CLASS_NAME, 'btn-submit').click()
    
# choose the criteria of searching
driver.find_element(By.CLASS_NAME, 'lnk-text').click()
time.sleep(3)
driver.find_element(By.ID, 'specific-sources-rd').click()
time.sleep(5)
    
# choose the journal
try:
    driver.find_element(By.ID, journal_code).click()
except:
    time.sleep(20)
    driver.find_element(By.ID, journal_code).click()
    
# key words : climat and climatique
driver.find_element(By.XPATH, '//textarea').send_keys('climat') 
driver.find_elements(By.CLASS_NAME, 'criteria-oper-lbl')[1].click()
Select(driver.find_element(By.NAME, 'CriteriaKeys[0].Key')).select_by_value('TEXT')
driver.find_element(By.NAME, 'CriteriaKeys[0].Text').send_keys('climatique')
    
# choose the searching period
Select(driver.find_element(By.ID, 'DateFilter_DateRange')).select_by_value('10')

Select(driver.find_elements(By.CLASS_NAME, 'year')[1]).select_by_value(start[0])
Select(driver.find_elements(By.CLASS_NAME, 'year')[2]).select_by_value(end[0])

Select(driver.find_elements(By.CLASS_NAME, 'month')[1]).select_by_value(start[1])
Select(driver.find_elements(By.CLASS_NAME, 'month')[2]).select_by_value(end[1])

Select(driver.find_elements(By.CLASS_NAME, 'day')[1]).select_by_value(start[2])
Select(driver.find_elements(By.CLASS_NAME, 'day')[2]).select_by_value(end[2])
    
driver.find_element(By.ID, 'btnSearch').click()
    
# scroll page
for i in range(15):
    temp_height=0
    while True:
        driver.execute_script("window.scrollBy(0,1000)")
        check_height = driver.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
        if check_height==temp_height:
            break
        temp_height=check_height
        
    time.sleep(5)
    if (i+1) % 5 == 0:
        print('scrolling ', i+1, ' times' )
        

<ipython-input-96-0ff624ea0ec2>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


scrolling  5  times
scrolling  10  times
scrolling  15  times


In [115]:
# get web contents
web_content = driver.page_source
obj = BeautifulSoup(web_content,'lxml').body
obj = obj.aside.ul
obj = obj.find_all(name='div', attrs={"class":"docListItem msDocItem"})
    
# start to build data
info = pd.DataFrame(columns = ['date', 'publication', 
                                   'words', 'author',
                                   'title', 'intro'])

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.96)
Stacktrace:
Backtrace:
	(No symbol) [0x009537D3]
	(No symbol) [0x008E8B81]
	(No symbol) [0x007EB36D]
	(No symbol) [0x007CE5DA]
	(No symbol) [0x0083F0CB]
	(No symbol) [0x00850A56]
	(No symbol) [0x0083B216]
	(No symbol) [0x00810D97]
	(No symbol) [0x0081253D]
	GetHandleVerifier [0x00BCABF2+2510930]
	GetHandleVerifier [0x00BF8EC1+2700065]
	GetHandleVerifier [0x00BFC86C+2714828]
	GetHandleVerifier [0x00A03480+645344]
	(No symbol) [0x008F0FD2]
	(No symbol) [0x008F6C68]
	(No symbol) [0x008F6D4B]
	(No symbol) [0x00900D6B]
	BaseThreadInitThunk [0x75B900F9+25]
	RtlGetAppContainerNamedObjectPath [0x77457BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77457B8E+238]


In [113]:
for i in range(len(obj)):
    obj_0 = obj[15]
    obj_0 = obj_0.div.next_sibling.next_sibling
    obj_0 = obj_0.div

<div class="docList-header"> <span class="icon-logo-container"><img src="/images/interface/result/ico_pressecrite_small.png" vspace="10"/></span> <span class="source-type pressecrite">Journaux</span> <input id="doc-name" type="hidden" value="news·20220917·LF·992×20×2190757176"/> <span class="source-name">Le Figaro</span> </div>

In [81]:
## Les Echos

journal_code = 'sf_251'
year_list = list(map(lambda x:str(x),list(range(2013,2023))))

import_data(journal_code, year_list)

['2022', '9', '1']

In [ ]:
## Libération

journal_code = 'sf_252'
year_list = list(map(lambda x:str(x),list(range(2013,2023))))

import_data(journal_code, year_list)

In [ ]:
## La Croix

journal_code = 'sf_243'
year_list = list(map(lambda x:str(x),list(range(2013,2023))))

import_data(journal_code, year_list)

In [ ]:
## Le Parisien

journal_code = 'sf_257'
year_list = list(map(lambda x:str(x),list(range(2013,2023))))

import_data(journal_code, year_list)